### Importação Bibliotecas

---

In [23]:
import mlflow
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time


### Constantes

---

In [24]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "default.payment.next.month"
URI = "http://localhost:5000"
TEST_SIZE = 0.2


O código abaixo define onde estará disponível o serviço de model tracking, neste caso o MLflow, que está a correr localmente na porta 5000 (http://localhost:5000). Este serviço será responsável por guardar os registos do treino dos modelos, incluindo parâmetros, métricas, artefactos e outros dados relevantes.

In [25]:
from pathlib import Path

mlflow.set_tracking_uri(URI)

Esta célula define o nome da experiência no MLflow nesse caso "Rumos Bank Lending". Através da experiência será possivel  agrupar todas as execuções (runs) relacionadas com um determinado objectivo ou projecto, facilitando a comparação de resultados entre diferentes modelos, configurações ou versões de treino.

In [26]:
mlflow.set_experiment("Rumos Bank Lending")

<Experiment: artifact_location='mlflow-artifacts:/395285683720049485', creation_time=1744151061274, experiment_id='395285683720049485', last_update_time=1744151061274, lifecycle_stage='active', name='Rumos Bank Lending', tags={}>

Esta célula carrega os dados a partir do ficheiro lending_data.csv, localizado no caminho definido por ROOT_PATH. Em seguida, seleciona aleatoriamente todas as linhas do DataFrame utilizando uma seed fixa (SEED) para garantir reprodutibilidade. 

In [27]:
df = pd.read_csv(ROOT_PATH + 'lending_data.csv')
df = df.sample(frac=0.01, random_state=SEED)

A célula abaixo divide o conjunto de dados df em dois subconjuntos: conjunto de treino (train_set) e conjunto de teste (test_set). A divisão é feita de forma aleatória, com 20% dos dados reservados para teste e os restantes 80% para treino. A utilização da seed (SEED) garante que a divisão é reprodutível — ou seja, será sempre igual em diferentes execuções do código. Esta separação é essencial para avaliar o desempenho dos modelos com dados que não foram usados durante o treino.

In [28]:
train_set, test_set = train_test_split(df, test_size = TEST_SIZE, random_state = SEED)

Esta célula separa as features (variáveis independentes) e o target (variável dependente) tanto para o conjunto de treino como para o conjunto de teste. A coluna 'default.payment.next.month' representa a variável que se pretende prever (se o cliente irá ou não entrar em incumprimento no mês seguinte), sendo por isso atribuída às variáveis alvo y_train e y_test. As restantes colunas são utilizadas como input (X_train e X_test) para treinar e testar os modelos.

In [29]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

### Guardar datasets, modelos, artefactos, métricas e parametros da run

---

Esta célula converte os conjuntos de treino e teste (train_set e test_set) em objectos mlflow.data, utilizando a função from_pandas. Isto permite que os dados sejam registados no MLflow de forma estruturada, com mmetadados adicionais, como a origem (source="split_from_full_dataset") e o nome do dataset.

In [30]:
# guardar o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")

c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigue

Esta célula define uma lista chamada models_and_params, que contém os pipelines e os respectivos espaços de procura de hiperparâmetros para seis modelos de machine learning: regressão logística, (KNN), árvore de decisão, (Random Forest), perceptrão multicamada (MLP) e (SVC).

Cada modelo é encapsulado num Pipeline que começa com um MinMaxScaler() para normalização das variáveis, garantindo que os dados de entrada estão na mesma escala — o que é especialmente importante para algoritmos baseados em distâncias ou gradientes.

Para cada modelo é também definido um dicionário com os hiperparâmetros a testar durante o processo de optimização (por exemplo, valores de C na regressão logística ou número de vizinhos no KNN). Esta estrutura será usada posteriormente com GridSearchCV para realizar a afinação automática dos modelos durante o treino.

In [31]:
models_and_params = [

    ("logistic_regression",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", LogisticRegression(max_iter=500, solver='lbfgs', random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__C": [0.001, 0.01, 0.1, 1, 10, 100]
     }),

    ("knn",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", KNeighborsClassifier())
     ]),
     {
         "Classifier__n_neighbors": list(range(1, 10))
     }),

    ("decision_tree",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", tree.DecisionTreeClassifier(random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__max_depth": [3, 6],
         "Classifier__min_samples_split": [2, 4, 10]
     }),

    ("random_forest",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", RandomForestClassifier(random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__n_estimators": [10, 100, 300, 1000]
     }),

    ("mlp",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", MLPClassifier(solver='lbfgs', random_state=SEED, max_iter=1000))
     ]),
     {
         "Classifier__hidden_layer_sizes": [(20,), (20, 10), (20, 10, 2)],
         "Classifier__learning_rate_init": [0.0001, 0.001, 0.01, 0.1]
     }),

    ("svc",
     Pipeline([
         ("scaler", MinMaxScaler()),
         ("Classifier", SVC(random_state=SEED, class_weight='balanced', gamma='scale', probability=True, verbose=True))
     ]),
     {
         "Classifier__C": [0.1, 1, 10],
         "Classifier__kernel": ['rbf', 'linear']
     })
]

O presente bloco de código percorre cada um dos modelos definidos anteriormente, treinando-os e registando-os no MLflow. Para cada modelo, inicia-se uma nova execução (run) no MLflow, onde são guardadas informações relevantes como os dados de treino e de teste (através dos objetos train_dataset e test_dataset), bem como a seed utilizada para garantir a reprodutibilidade.

De seguida, é utilizado o GridSearchCV com validação cruzada de 5 folds para encontrar a melhor combinação de hiperparâmetros com base na métrica de accuracy. Após o treino, é registado o tempo total de execução e identificado o melhor modelo encontrado.

Este modelo é então guardado no MLflow, e as respetivas experiências. Por fim, são também registados os melhores parâmetros encontrados, a pontuação média obtida na validação cruzada e o tempo de treino em segundos. Este processo permite treinar e comparar vários modelos de forma estruturada e facilmente rastreável.

In [32]:
for model_name, pipeline, param_grid in models_and_params:

    # Iniciar run no MlFlow
    run = mlflow.start_run(run_name="Ml Model Run "+model_name+ "- pipeline")
    RUN_ID = run.info.run_uuid

    
    # Guardar Informação Dataset de treino e de teste associado à run
    mlflow.log_input(train_dataset, context="train")
    mlflow.log_input(test_dataset, context="test")

    # Guardar parametros seed  e text_size utilizado como parametro
    mlflow.log_param("seed", SEED)
    mlflow.log_param("test size", TEST_SIZE)


    print(f"Treinando modelo e guardando todos os logs em MlFlow: {model_name}")
    start_time = time.time()
    
    # GridSearchCV para encontrar os melhores parâmetros
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)


    end_time = time.time()
    elapsed_time = end_time - start_time  # tempo em segundos

    best_model = grid_search.best_estimator_

    #Guardar Log do modelo no MLflow
    mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path=model_name,
            registered_model_name=model_name
    )

    # Guardar Log dos parâmetros e métricas do modelo
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("best_cv_score", grid_search.best_score_)
    mlflow.log_metric("training_time_sec", elapsed_time)  # tempo registado

    print(f"Modelo '{model_name}' guardado  MLflow.")
    mlflow.end_run()

c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Treinando modelo e guardando todos os logs em MlFlow: logistic_regression


2025/04/10 17:55:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'logistic_regression' already exists. Creating a new version of this model...
2025/04/10 17:55:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regression, version 25
Created version '25' of model 'logistic_regression'.


Modelo 'logistic_regression' guardado  MLflow.
🏃 View run Ml Model Run logistic_regression- pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/50462057b45e4a18acfcc76f7e103a94
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: knn


2025/04/10 17:55:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'knn' already exists. Creating a new version of this model...
2025/04/10 17:55:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: knn, version 25
Created version '25' of model 'knn'.


...

2025/04/10 17:56:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'mlp' already exists. Creating a new version of this model...
2025/04/10 17:57:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mlp, version 17
Created version '17' of model 'mlp'.


Modelo 'mlp' guardado  MLflow.
🏃 View run Ml Model Run mlp- pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/80aa1f603c154e178f7de5d80fd0cb82
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: svc
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

2025/04/10 17:57:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'svc' already exists. Creating a new version of this model...
2025/04/10 17:57:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: svc, version 10


Modelo 'svc' guardado  MLflow.
🏃 View run Ml Model Run svc- pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/27c94d504bb846bca7461cd030716f24
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485


Created version '10' of model 'svc'.
